**Import libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

import sklearn
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


**Load the dataset**

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [5]:
# Load dataset
def load_data():
    data =pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/tweet_covid_processing_file.csv", engine='python')
    return data

In [6]:
tweet_df = load_data()
tweet_df.head(5)

,Unnamed: 0,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,0.0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,1.0,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,2.0,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,3.0,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,4.0,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [7]:
tweet_df.drop('Unnamed: 0',inplace=True,axis = 1)

In [8]:
tweet_df.head(5)

,OriginalTweet,Sentiment,label,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,0,MeNyrbie Phil Gahan Chrisitv and and,"['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv', 'and...","['MeNyrbie', 'Phil', 'Gahan', 'Chrisitv']",menyrbi phil gahan chrisitv,MeNyrbie Phil Gahan Chrisitv
1,advice Talk to your neighbours family to excha...,Positive,1,advice Talk to your neighbours family to excha...,"['advice', 'Talk', 'to', 'your', 'neighbours',...","['advice', 'Talk', 'neighbours', 'family', 'ex...",advic talk neighbour famili exchang phone numb...,advice Talk neighbour family exchange phone nu...
2,Coronavirus Australia: Woolworths to give elde...,Positive,1,Coronavirus Australia Woolworths to give elde...,"['Coronavirus', 'Australia', 'Woolworths', 'to...","['Coronavirus', 'Australia', 'Woolworths', 'gi...",coronaviru australia woolworth give elderli di...,Coronavirus Australia Woolworths give elderly ...
3,My food stock is not the only one which is emp...,Positive,1,My food stock is not the only one which is emp...,"['My', 'food', 'stock', 'is', 'not', 'the', 'o...","['My', 'food', 'stock', 'one', 'empty', 'PLEAS...",My food stock one empti pleas panic there will...,My food stock one empty PLEASE panic THERE WIL...
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,3,Me ready to go at supermarket during the COV...,"['Me', 'ready', 'to', 'go', 'at', 'supermarket...","['Me', 'ready', 'go', 'supermarket', 'COVID', ...",Me readi go supermarket covid outbreak not I p...,Me ready go supermarket COVID outbreak Not I p...


In [9]:
tweet_df.isnull().sum()

OriginalTweet            0
Sentiment                2
label                    2
clean_tweet              2
tweet_token              2
tweet_token_filtered     2
tweet_stemmed           16
tweet_lemmatized        18
dtype: int64

**Drop Null values**

In [10]:
tweet_df= tweet_df.dropna()

In [11]:
tweet_df.isnull().sum()

OriginalTweet           0
Sentiment               0
label                   0
clean_tweet             0
tweet_token             0
tweet_token_filtered    0
tweet_stemmed           0
tweet_lemmatized        0
dtype: int64

In [28]:
len(tweet_df['tweet_lemmatized'])

41141

In [37]:
tweet_df['tweet_lemmatized'][1]

'advice Talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist GP set online shopping account po adequate supply regular med order'

**Train the Word2Vec using gensim**

In [46]:
# Creating the model and setting values for the various parameters
num_features = 200  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences=tweet_df['tweet_lemmatized'],\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

Training model....


In [47]:
words = list(model.wv.vocab)
print('Here is the Vocabulary Size.. %d' % len(words))
words

Here is the Vocabulary Size.. 53


['M',
 'e',
 'N',
 'y',
 'r',
 'b',
 'i',
 ' ',
 'P',
 'h',
 'l',
 'G',
 'a',
 'n',
 'C',
 's',
 't',
 'v',
 'd',
 'c',
 'T',
 'k',
 'g',
 'o',
 'u',
 'f',
 'm',
 'x',
 'p',
 'q',
 'A',
 'W',
 'w',
 'O',
 'V',
 'I',
 'D',
 'L',
 'E',
 'S',
 'H',
 'R',
 'B',
 'U',
 'F',
 'Y',
 'z',
 'j',
 'Z',
 'J',
 'K',
 'X',
 'Q']

In [48]:
model.wv.most_similar("A")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('E', 0.8449580669403076),
 ('R', 0.7846043705940247),
 ('S', 0.7691285610198975),
 ('G', 0.7684274315834045),
 ('F', 0.7339881658554077),
 ('N', 0.722922682762146),
 ('P', 0.7191516160964966),
 ('L', 0.6974988579750061),
 ('M', 0.6945151090621948),
 ('H', 0.667276918888092)]

In [50]:
# Importing bokeh libraries for showing how words of similar context are grouped together
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
import matplotlib.pyplot as plt

voc_size=3000
#Defining the chart
output_notebook()
plot_chart = bp.figure(plot_width=700, plot_height=600, title="A map/plot of 3000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    x_axis_type=None, y_axis_type=None, min_border=1)

#Extracting the list of word vectors, limiting to 1000, each is of 200 dimensions
word_vectors = [model[w] for w in list(model.wv.vocab.keys())[:voc_size]]

# Reducing dimensionality by converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# Storing data in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(model.wv.vocab.keys())[:voc_size]

# Corresponding word appears when you hover on the data point.
plot_chart.scatter(x='x', y='y', source=tsne_df)
hover = plot_chart.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_chart)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


[t-SNE] Computing 52 nearest neighbors...
[t-SNE] Indexed 53 samples in 0.000s...
[t-SNE] Computed neighbors for 53 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 53 / 53
[t-SNE] Mean sigma: 0.733003
[t-SNE] KL divergence after 250 iterations with early exaggeration: 47.658154
[t-SNE] KL divergence after 1000 iterations: 0.351829


In [51]:
"""import numpy as np
embedding_matrix = np.zeros((len(model.wv.vocab) + 1, 300))
for i, vec in enumerate(model.wv.vectors):
    embedding_matrix[i] = vec"""
import numpy as np 
#Build word vector set by using the average value of all word vectors , then scale
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size)) #As word vectors are of zero length size value(i.e 300) 
    count = 0 # no. of words with a valid vector in the sentence/review
    for word in text: #for each word in a sentence/review
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [52]:
from sklearn.preprocessing import scale
vecs = np.concatenate([buildWordVector(z, Size) for z in tweet_df['tweet_lemmatized']])
#print("Before Scaling:",vecs[1:2])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [53]:
print("Dimension of vector :",vecs.shape)

Dimension of vector : (41141, 200)


In [54]:
vecs

array([[ 0.01455165,  0.0375589 ,  0.02465053, ..., -0.00917693,
         0.00718173, -0.00288007],
       [-0.00586966,  0.06729438,  0.05745566, ..., -0.01253773,
        -0.00488357, -0.01230897],
       [-0.0049429 ,  0.05172257,  0.05570694, ...,  0.00800712,
        -0.00479591, -0.01645128],
       ...,
       [-0.0048766 ,  0.07091722,  0.05119136, ..., -0.0058158 ,
        -0.00689215, -0.01608963],
       [ 0.00090541,  0.0433938 ,  0.06198582, ...,  0.01303102,
        -0.01470601, -0.01638389],
       [-0.00147304,  0.0571238 ,  0.04876145, ..., -0.01331844,
        -0.00956458, -0.00443603]])

In [55]:
import sklearn
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler

In [57]:
pip install opentsne

     |████████████████████████████████| 1.8MB 969kB/s 


In [58]:
from openTSNE import TSNE

tsne_model_v2 = tsne = TSNE(
    n_components=2, perplexity=30, learning_rate=200,
    n_jobs=4, initialization="pca", metric="euclidean",
    early_exaggeration_iter=250, early_exaggeration=12, n_iter=750,
    neighbors="exact", negative_gradient_method="bh")

vecs_v1 = tsne_model_v2.fit(vecs)

In [59]:
vecs_v1.shape

(41141, 2)

In [60]:

Y_word2vec = tweet_df['label']

In [61]:
smk_tiagram = SMOTETomek(random_state=42)
X_vec_v1,Y_vec_v1=smk_tiagram.fit_sample(vecs_v1,Y_word2vec)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [62]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y_word2vec)))
print('Resampled dataset shape {}'.format(Counter(Y_vec_v1)))

Original dataset shape Counter({'1': 11422, '4': 9917, '0': 7697, '2': 6624, '3': 5481})
Resampled dataset shape Counter({'3': 8360, '2': 8163, '0': 8030, '4': 7293, '1': 6944})


In [63]:
# let's see how well our model perform on this new data
x_train,x_test,y_train,y_test = train_test_split(X_vec_v1,Y_vec_v1,test_size = 0.30, random_state= True)

**Made the classifier**

In [ ]:
#We shall experiment common traditional machine learning algorithms in this section. We have done already for extracting the features using Word2Vec lirary

In [64]:
# Decission Tree Classifier
decession_tree = DecisionTreeClassifier()
decession_tree.fit(x_train,y_train)
y_pred_decession_tree = decession_tree.predict(x_test)
print("Decission Tree Classifier")
print(classification_report(y_test, y_pred_decession_tree))

# Random Forest  Classifier
random_forest = RandomForestClassifier()
random_forest.fit(x_train,y_train)
y_pred_random_forest = random_forest.predict(x_test)
print("Random Forest  Classifier")
print(classification_report(y_test, y_pred_random_forest))


# KNN  Algo
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(x_train,y_train)
y_pred_knn_classifier = knn_classifier.predict(x_test)
print("KNN  Algo")
print(classification_report(y_test, y_pred_knn_classifier))


# Multinomial Naive Bias
mulNB = GaussianNB()
mulNB.fit(x_train,y_train)
y_pred_mulNB = mulNB.predict(x_test)
print("Gaussian Naive Bias")
print(classification_report(y_test, y_pred_mulNB))

Decission Tree Classifier
              precision    recall  f1-score   support

           0       0.48      0.50      0.49      2376
           1       0.34      0.30      0.32      2108
           2       0.47      0.48      0.48      2403
           3       0.51      0.54      0.53      2525
           4       0.38      0.37      0.37      2225

    accuracy                           0.44     11637
   macro avg       0.44      0.44      0.44     11637
weighted avg       0.44      0.44      0.44     11637

Random Forest  Classifier
              precision    recall  f1-score   support

           0       0.54      0.58      0.56      2376
           1       0.37      0.30      0.33      2108
           2       0.51      0.56      0.53      2403
           3       0.54      0.61      0.57      2525
           4       0.45      0.39      0.41      2225

    accuracy                           0.49     11637
   macro avg       0.48      0.48      0.48     11637
weighted avg       0.48  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
# Gradient bossting
gradient_bossting = GradientBoostingClassifier(random_state=3)
gradient_bossting.fit(x_train,y_train)
y_pred_gradientbossting = gradient_bossting.predict(x_test)
print("GradientBosting  Algo")
print(classification_report(y_test, y_pred_gradientbossting))


# SVM
svm_class = svm.SVC(kernel='poly')
svm_class.fit(x_train,y_train)
y_pred_svm = svm_class.predict(x_test)
print("SVM  Algo")
print(classification_report(y_test, y_pred_svm))


# Logistic Regressor
logis_reg =LogisticRegression()
logis_reg.fit(x_train,y_train)
y_pred_log = logis_reg.predict(x_test)
print("Logistic Reg  Algo")
print(classification_report(y_test, y_pred_log))

GradientBosting  Algo
              precision    recall  f1-score   support

           0       0.37      0.45      0.41      2376
           1       0.28      0.02      0.03      2108
           2       0.27      0.49      0.35      2403
           3       0.28      0.44      0.34      2525
           4       0.31      0.04      0.07      2225

    accuracy                           0.30     11637
   macro avg       0.30      0.29      0.24     11637
weighted avg       0.30      0.30      0.25     11637

SVM  Algo


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.34      0.40      0.37      2376
           1       0.00      0.00      0.00      2108
           2       0.27      0.42      0.33      2403
           3       0.24      0.48      0.32      2525
           4       0.00      0.00      0.00      2225

    accuracy                           0.27     11637
   macro avg       0.17      0.26      0.20     11637
weighted avg       0.18      0.27      0.21     11637

Logistic Reg  Algo
              precision    recall  f1-score   support

           0       0.30      0.57      0.39      2376
           1       0.00      0.00      0.00      2108
           2       0.27      0.49      0.34      2403
           3       0.26      0.28      0.27      2525
           4       0.00      0.00      0.00      2225

    accuracy                           0.28     11637
   macro avg       0.17      0.27      0.20     11637
weighted avg       0.17      0.28      0.21     11637



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
